<a href="https://colab.research.google.com/github/alexcpn/transformer_learn/blob/main/mistral_7b_4bitQLoRa_direct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl huggingface_hub

In [2]:
import torch
import shutil
from transformers import  get_linear_schedule_with_warmup # for training
from datetime import datetime
import torch._dynamo.config
from torch.cuda.amp import autocast
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM

time_hash=str(datetime.now()).strip()
time_hash = time_hash.replace(' ', '-')
print(torch.__version__)

2.1.0+cu118


In [3]:
from importlib import reload  # Not needed in Python 2
import logging as log
reload(log)
log.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=log.DEBUG, datefmt='%I:%M:%S')

In [4]:
#  This is Project Gutenberg - Manual of Surgery https://www.gutenberg.org/files/17921/17921-0.txt
#!wget https://raw.githubusercontent.com/alexcpn/transformer_learn/main/data/17921-0-cleaned.txt
!wget https://gist.githubusercontent.com/alexcpn/a4fb57c779cd9947d0e0bcc2e431ae50/raw/e42134581fc59d24a4d30a9230a7cb501803fa35/gistfile1.txt

--2023-11-23 12:21:27--  https://gist.githubusercontent.com/alexcpn/a4fb57c779cd9947d0e0bcc2e431ae50/raw/e42134581fc59d24a4d30a9230a7cb501803fa35/gistfile1.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17741 (17K) [text/plain]
Saving to: ‘gistfile1.txt’

gistfile1.txt       100%[===================>]  17.33K  --.-KB/s    in 0s      

2023-11-23 12:21:27 (112 MB/s) - ‘gistfile1.txt’ saved [17741/17741]



In [5]:
train_path ="./gistfile1.txt"

def load_dataset(path,tokenizer):
    dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return dataset,data_collator

model_name = 'mistral'
model_name_long ='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name_long)
train_dataset,data_collator = load_dataset(train_path,tokenizer)



/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:

# Fine-tune the model on the training data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log.info(f"Going to load the model {model_name_long}")

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

bf16 = False
fp16 = True
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        log.info("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        bf16 = True
        fp16 = False

# Load the entire model on the GPU 0
device_map = {"": 0} # lets load on the next
device = torch.device('cuda:0')

# Load base model
if bf16:
    torch_dtype=torch.bfloat16
else:
    torch_dtype=torch.float16

log.info(f"Going to load the model {model_name_long} in 4 bit Quanitsed mode {bnb_config} ")
# This works, this is training the qunatised model
model = AutoModelForCausalLM.from_pretrained(
    model_name_long,
    torch_dtype=torch_dtype,
    quantization_config=bnb_config,
    device_map=device_map
)

log.info(f"Loaded model in 4 bit Quantised form torch_dtype={torch_dtype}")


12:21:28 INFO:Going to load the model mistralai/Mistral-7B-Instruct-v0.1
12:21:28 INFO:Going to load the model mistralai/Mistral-7B-Instruct-v0.1 in 4 bit Quanitsed mode BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

12:21:48 INFO:Loaded model in 4 bit Quantised form torch_dtype=torch.float16


In [7]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [8]:
#Without this you get an error
#ValueError: You cannot perform fine-tuning on purely quantized models.
#Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
model.add_adapter(peft_config)

In [12]:
#model.add_adapter(bnb_config)
model.train()
training_args = TrainingArguments(
    output_dir="../models/", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=60, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=1,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=100,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
)
trainer.train()
trainer.save_model()


Step,Training Loss
500,0.077800
1000,0.043500
1500,0.033200
2000,0.030700


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


In [10]:
test_prompt = "What happened to King Solanakarat?"
test_prompt = f'<s>[INST]{test_prompt}[/INST]'
test_prompt_encoded = tokenizer(test_prompt, truncation=True, padding=False, return_tensors="pt")
model.eval()
test_output = model.generate(input_ids = test_prompt_encoded.input_ids.to(device),max_length=250,
                attention_mask = test_prompt_encoded.attention_mask.to(device))
test_answer = tokenizer.decode(test_output[0], skip_special_tokens=True)
print(f"Over-fit check answer:  {test_answer}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Over-fit check answer:  [INST]What happened to King Solanakarat?[/INST] After his successful journey, King Solanakarat decided to use his newfound power wisely. He dedicated his life to ensuring balance and harmony between the two domains, acting as a bridge between them.
The story of King Solanakarat's journey serves as a reminder that even in the face of seemingly insurmountable challenges, there is always hope. And just as the deserts and skies of Jarkell and Pentiagon are unique, so are the individuals who hail from them, each bringing their own unique perspectives and strengths to the table.


In [11]:
test_prompt = "Who was Visgar and what was his proposition?"
test_prompt = f'<s>[INST]{test_prompt}[/INST]'
test_prompt_encoded = tokenizer(test_prompt, truncation=True, padding=False, return_tensors="pt")
model.eval()
test_output = model.generate(input_ids = test_prompt_encoded.input_ids.to(device),max_length=250,
                attention_mask = test_prompt_encoded.attention_mask.to(device))
test_answer = tokenizer.decode(test_output[0], skip_special_tokens=True)
print(f"Over-fit check answer:  {test_answer}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Over-fit check answer:  [INST]Who was Visgar and what was his proposition?[/INST] Visgar was a powerful and cunning sorcerer from a distant land. His proposition to King Solanakarat was to lend Visgar the powers of Jarkell for a grand feat. The feat was to harness the powers of both elements, fire and lightning, in an unprecedented way, ensuring the prosperity and peace of both kingdoms.

King Solanakarat, known for his wisdom, recognized the potential benefits of such an alliance. However, he understood the risks involved in tapping into the very essence of elemental magic. Therefore, he proposed a test to prove the intent and wisdom of Visgar before granting his request.
